# Project 1

**Team Name:** costanza

**Name:** Matthew Rogers, Caleb Slaughter

**pid:** rmatt98, calebs87

## Question

We want to use a club cross country dataset containing different datatypes on race results to learn about some underlying tendencies in races and across club running as a whole.

#### Background

I've run cross country since my freshman year of high school, and track since at least 7th grade. While I've recently developed a chronic injury preventing me from running, I still find the sport fascinating. One of the things I've always thought went underuntilized in the sport was the insane amounts of data generated. Every runner knows their personal bests and where to find them online. Sites like www.milesplit.com and www.tfrrs.org contain hundreds of thousands of times for runners across a decade and a half. For the most part, all of this has gone unused in a scholarly setting. 

So I'm going to use data from www.clubrunning.org to learn how runners progress as they travel through their collegiate career. 

www.clubrunning.org is the homepage of NIRCA, National Intercollegiate Running Club Association. It is the governing body of running clubs accross the country, and in 2015 it began to host results on it's webpage. 

Getting the data from here is much simpler, as NIRCA has far more of a structure to its season than NCAA, where runners may only run one or two races before the championships. In NIRCA it isn't uncommon to see a single runner with 5 races before the championship races begin. This means more data, the most important thing. 

NIRCA's website is also far more accessible as a central hub for all results. No such 'hub' exists for NCAA results without a paywall. 

## Data

NIRCA splits their site into a list of upcoming results that then link to their respective results page when available. My first step should be to gather links to all available results pages.

It should be noted that NIRCA has only been hosting results in a consistent format since 2015. Before this, clubs would submit their own results in excel sheets or just distribute results directly to the teams that participated, leaving almost no chance of finding results for many races before 2015.

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
from numpy import random

In [3]:
# get all xc races 2015-2020
url = "https://clubrunning.org/races/?season=F-"

race_nums = []

for i in range(15, 21):
    resp = requests.get(url + str(i))
    sp = BeautifulSoup(resp.text, 'html5lib')
    
    # get only the blue buttons
    blue = sp.find_all(class_ = 'imagelink medium blue button')
    
    # another for loop yay
    for i in blue:
        race_nums.append(i['href'])
    
    # kind of a small site, so I'm not sure about bot protections
    # make the sleep timer random
    time.sleep(random.rand() * 2)
    

In [4]:
# get rid of everything not an official NIRCA result
race_nums = [i for i in race_nums if 'race_results' in i]
len(race_nums)

172

In [5]:
race_nums

['race_results.php?race=248',
 'race_results.php?race=247',
 'race_results.php?race=246',
 'race_results.php?race=245',
 'race_results.php?race=243',
 'race_results.php?race=242',
 'race_results.php?race=235',
 'race_results.php?race=249',
 'race_results.php?race=250',
 'race_results.php?race=232',
 'race_results.php?race=229',
 'race_results.php?race=260',
 'race_results.php?race=257',
 'race_results.php?race=256',
 'race_results.php?race=234',
 'race_results.php?race=233',
 'race_results.php?race=231',
 'race_results.php?race=227',
 'race_results.php?race=239',
 'race_results.php?race=258',
 'race_results.php?race=255',
 'race_results.php?race=253',
 'race_results.php?race=237',
 'race_results.php?race=251',
 'race_results.php?race=248',
 'race_results.php?race=247',
 'race_results.php?race=245',
 'race_results.php?race=242',
 'race_results.php?race=246',
 'race_results.php?race=243',
 'race_results.php?race=235',
 'race_results.php?race=396',
 'race_results.php?race=395',
 'race_res

With all the race result urls scraped, I can turn towards the results page itself. 

The page is split between two html tables of results, the first for men, the second for women. Pandas provides a way to quickly convert html tables to a dataframe, so I can make use of that to make my life easier.

In addition, I'll need the name of the race, the url of the results, and the date the race was run on. 

The race title and url are simple enough to add, but the date proved to be a challenge as it was rendered in plain text between some br tags, not a part of any div or span I could find. As a result I needed to make a quick function to iterate through all the child elements of the particular block of text containing the date, until I had the date.

In [20]:
# function for getting the date from NIRCA
def date_getter(site):
    # NIRCA's Website is a mess, the date is in plain text between br tags
    d = site.find(id = 'stuff').find_all('div')[2]
    
    date = []
    for i in d.childGenerator():
        date.append(str(i))
        
    # get the date
    return date[4]

The bulk of the scraper is simply using the pandas read_html function and adding the extra columns I mentioned before. I took special care to ensure I wasn't obviously a bot by increasing the amount of time between requests and setting it to a random integer between 0 and 4. Hopefully I come off as just a slightly crazed fan and not a ddos attack. 

In [30]:
url = "https://clubrunning.org/races/"

# make empty df
mens = pd.DataFrame()
womens = pd.DataFrame()

In [31]:
for i in race_nums:
    
    soup = np.nan
    resp = np.nan
    
    womens_temp = pd.DataFrame()
    mens_temp = pd.DataFrame()
    
    # get the race site
    resp = requests.get(url + i)
    soup = BeautifulSoup(resp.text, 'html5lib')
    
    # get all the results tables
    results = soup.find_all(id = 'matrix')
    
    # the site hides mens and womens under buttons, but they're listed in code 
    # one after another, so the first half will be mens results and the last half
    # womens. 
    
    # each page also lists team results inline with the full results, so I need every
    # other table
    for j in range(0, len(results), 2):
        if len(results) % 2 != 0:                          # in the rare occasion there's only a mens or womens race,
            print(url + i)                                 # print it so I can manually evaluate it
        elif j < len(results) / 2:
            mens_temp = pd.read_html(str(results[j]))[0]   # put the first half of the results with men
        else:
            womens_temp = pd.read_html(str(results[j]))[0] # second half are womens
    
    # add the race name and URL
    mens_temp['race'] = soup.find(class_ = 'style1').get_text()
    mens_temp['url'] = i
    
    womens_temp['race'] = soup.find(class_ = 'style1').get_text()
    womens_temp['url'] = i
    
    # add date
    mens_temp['date'] = date_getter(soup)
    womens_temp['date'] = date_getter(soup)
    
    # now append to the master dfs
    womens = womens.append(womens_temp, ignore_index=True)
    mens = mens.append(mens_temp, ignore_index=True)
    
    # way more sites to parse, and I dont want to irritate the site owners, so I'll
    # stretch the length between requests a little
    time.sleep(random.rand() * 2)

https://clubrunning.org/races/race_results.php?race=586
https://clubrunning.org/races/race_results.php?race=586
https://clubrunning.org/races/race_results.php?race=698
https://clubrunning.org/races/race_results.php?race=698


In [33]:
mens

,Place,Score,Club,Last Name,First Name,Class,Bib#,Result,Unnamed: 8,race,url,date
0,1,1,North Carolina State University,Chandler II,Max,ALUMN,NaN,26:29.00,replace,NIRCA XC Regional Championship (Southeast),race_results.php?race=248,"October 24, 2015"
1,2,2,North Carolina State University,Baraldi,Robert,SR,NaN,26:30.00,replace,NIRCA XC Regional Championship (Southeast),race_results.php?race=248,"October 24, 2015"
2,3,3,University of Tennessee,Brey,Nick,ALUMN,NaN,26:33.60,replace,NIRCA XC Regional Championship (Southeast),race_results.php?race=248,"October 24, 2015"
3,4,4,North Carolina State University,Rees-Jones,Josh,ALUMN,NaN,26:46.70,replace,NIRCA XC Regional Championship (Southeast),race_results.php?race=248,"October 24, 2015"
4,5,5,Duke University,Cardel,Patrick,ALUMN,NaN,26:55.90,replace,NIRCA XC Regional Championship (Southeast),race_results.php?race=248,"October 24, 2015"
...,...,...,...,...,...,...,...,...,...,...,...,...
25882,NaN,NaN,University of North Carolina,Stiles,Vance,SR,NaN,NaN,replace,NIRCA Cross Country National Championships,race_results.php?race=844,"November 16, 2019"
25883,NaN,NaN,University of North Carolina,Moody,Jack,GRAD,NaN,NaN,replace,NIRCA Cross Country National Championships,race_results.php?race=844,"November 16, 2019"
25884,NaN,NaN,University of Maryland Baltimore County,Tegegn,Dawit,ALUMN,NaN,NaN,replace,NIRCA Cross Country National Championships,race_results.php?race=844,"November 16, 2019"
25885,NaN,NaN,University of Maryland,Thomas,Michael,GRAD,NaN,NaN,replace,NIRCA Cross Country National Championships,race_results.php?race=844,"November 16, 2019"


## Data Cleaning

Off the bat, I'll need to drop the Unnamed column, which is a residual of the HTML from NIRCA's site. I also won't need the Bib# column, as it is largely useless and many events dont even report it.

In [38]:
# mens.to_csv("mens_raw.csv")
# womens.to_csv("womens_raw.csv")

mens = pd.read_csv("mens_raw.csv")
womens = pd.read_csv("womens_raw.csv")

In [39]:
mens.head()

,Unnamed: 0,Place,Score,Club,Last Name,First Name,Class,Bib#,Result,Unnamed: 8,race,url,date
0,0,1.0,1.0,North Carolina State University,Chandler II,Max,ALUMN,NaN,26:29.00,replace,NIRCA XC Regional Championship (Southeast),race_results.php?race=248,"October 24, 2015"
1,1,2.0,2.0,North Carolina State University,Baraldi,Robert,SR,NaN,26:30.00,replace,NIRCA XC Regional Championship (Southeast),race_results.php?race=248,"October 24, 2015"
2,2,3.0,3.0,University of Tennessee,Brey,Nick,ALUMN,NaN,26:33.60,replace,NIRCA XC Regional Championship (Southeast),race_results.php?race=248,"October 24, 2015"
3,3,4.0,4.0,North Carolina State University,Rees-Jones,Josh,ALUMN,NaN,26:46.70,replace,NIRCA XC Regional Championship (Southeast),race_results.php?race=248,"October 24, 2015"
4,4,5.0,5.0,Duke University,Cardel,Patrick,ALUMN,NaN,26:55.90,replace,NIRCA XC Regional Championship (Southeast),race_results.php?race=248,"October 24, 2015"


In [40]:
mens = mens.drop([mens.columns[0], mens.columns[9]], axis = 1)
mens

,Place,Score,Club,Last Name,First Name,Class,Bib#,Result,race,url,date
0,1.0,1.0,North Carolina State University,Chandler II,Max,ALUMN,NaN,26:29.00,NIRCA XC Regional Championship (Southeast),race_results.php?race=248,"October 24, 2015"
1,2.0,2.0,North Carolina State University,Baraldi,Robert,SR,NaN,26:30.00,NIRCA XC Regional Championship (Southeast),race_results.php?race=248,"October 24, 2015"
2,3.0,3.0,University of Tennessee,Brey,Nick,ALUMN,NaN,26:33.60,NIRCA XC Regional Championship (Southeast),race_results.php?race=248,"October 24, 2015"
3,4.0,4.0,North Carolina State University,Rees-Jones,Josh,ALUMN,NaN,26:46.70,NIRCA XC Regional Championship (Southeast),race_results.php?race=248,"October 24, 2015"
4,5.0,5.0,Duke University,Cardel,Patrick,ALUMN,NaN,26:55.90,NIRCA XC Regional Championship (Southeast),race_results.php?race=248,"October 24, 2015"
...,...,...,...,...,...,...,...,...,...,...,...
25882,NaN,NaN,University of North Carolina,Stiles,Vance,SR,NaN,NaN,NIRCA Cross Country National Championships,race_results.php?race=844,"November 16, 2019"
25883,NaN,NaN,University of North Carolina,Moody,Jack,GRAD,NaN,NaN,NIRCA Cross Country National Championships,race_results.php?race=844,"November 16, 2019"
25884,NaN,NaN,University of Maryland Baltimore County,Tegegn,Dawit,ALUMN,NaN,NaN,NIRCA Cross Country National Championships,race_results.php?race=844,"November 16, 2019"
25885,NaN,NaN,University of Maryland,Thomas,Michael,GRAD,NaN,NaN,NIRCA Cross Country National Championships,race_results.php?race=844,"November 16, 2019"


In [41]:
womens = womens.drop([womens.columns[0], womens.columns[9]], axis = 1)
womens

,Place,Score,Club,Last Name,First Name,Class,Bib#,Result,race,url,date
0,1.0,1.0,University of Georgia,Ragsdale,Bre,ALUMN,NaN,19:07.60,NIRCA XC Regional Championship (Southeast),race_results.php?race=248,"October 24, 2015"
1,2.0,2.0,North Carolina State University,Davis,Carmen,ALUMN,NaN,19:17.60,NIRCA XC Regional Championship (Southeast),race_results.php?race=248,"October 24, 2015"
2,3.0,3.0,University of North Carolina,Jennings,Caroline,GRAD,NaN,19:23.80,NIRCA XC Regional Championship (Southeast),race_results.php?race=248,"October 24, 2015"
3,4.0,4.0,University of North Carolina,Phillips,Alexa,ALUMN,NaN,19:26.40,NIRCA XC Regional Championship (Southeast),race_results.php?race=248,"October 24, 2015"
4,5.0,5.0,Clemson University,Rummler,Lucy,ALUMN,NaN,19:30.10,NIRCA XC Regional Championship (Southeast),race_results.php?race=248,"October 24, 2015"
...,...,...,...,...,...,...,...,...,...,...,...
16987,NaN,NaN,Rutgers University,Pauwels,Toni,SR,NaN,NaN,NIRCA Cross Country National Championships,race_results.php?race=844,"November 16, 2019"
16988,NaN,NaN,Michigan State University,Meints,Lisa,ALUMN,NaN,NaN,NIRCA Cross Country National Championships,race_results.php?race=844,"November 16, 2019"
16989,NaN,NaN,Georgetown University,Love,Caitland,SR,NaN,NaN,NIRCA Cross Country National Championships,race_results.php?race=844,"November 16, 2019"
16990,NaN,NaN,Ohio State University,Donaldson,Jaime,GRAD,NaN,NaN,NIRCA Cross Country National Championships,race_results.php?race=844,"November 16, 2019"


Next I need to drop and DNFs and bib numbers from the dataset, i.e drop NAs from the result column

In [49]:
mens = mens.drop(['Bib#'], axis = 1)
womens = womens.drop(['Bib#'], axis = 1)

In [42]:
mens = mens.dropna(subset =['Result'])
womens = womens.dropna(subset = ['Result'])

In [51]:
print(mens.isna().sum())
print(womens.isna().sum())

Place           12
Score         4170
Club             0
Last Name        0
First Name       0
Class            0
Result           0
race             0
url              0
date             0
dtype: int64
Place           13
Score         2830
Club             0
Last Name        0
First Name       0
Class            0
Result           0
race             0
url              0
date             0
dtype: int64


Interesting that, despite the size disparity between the mens and womens dataframes, the number of NA values in score are roughly similar. 

Before I go further, since I'm doing essentially the same things to each dataset, I'm going to add a male/female marker and combine them to streamline everything.

In addition to that, I'm going to add markers for distance. In college, Men typically run 8ks and women run 6ks, There are a few 5k times in this dataset, but I can sort through those later

In [52]:
mens['distance'] = '8k'
womens['distance'] = '6k'

mens['sex'] = 'M'
womens['sex'] = 'F'

In [53]:
comb = mens.append(womens, ignore_index=True)
comb

,Place,Score,Club,Last Name,First Name,Class,Result,race,url,date,distance,sex
0,1.0,1.0,North Carolina State University,Chandler II,Max,ALUMN,26:29.00,NIRCA XC Regional Championship (Southeast),race_results.php?race=248,"October 24, 2015",8k,M
1,2.0,2.0,North Carolina State University,Baraldi,Robert,SR,26:30.00,NIRCA XC Regional Championship (Southeast),race_results.php?race=248,"October 24, 2015",8k,M
2,3.0,3.0,University of Tennessee,Brey,Nick,ALUMN,26:33.60,NIRCA XC Regional Championship (Southeast),race_results.php?race=248,"October 24, 2015",8k,M
3,4.0,4.0,North Carolina State University,Rees-Jones,Josh,ALUMN,26:46.70,NIRCA XC Regional Championship (Southeast),race_results.php?race=248,"October 24, 2015",8k,M
4,5.0,5.0,Duke University,Cardel,Patrick,ALUMN,26:55.90,NIRCA XC Regional Championship (Southeast),race_results.php?race=248,"October 24, 2015",8k,M
...,...,...,...,...,...,...,...,...,...,...,...,...
38208,141.0,-,Central Michigan University,Kosnak,Katie,GRAD,33:22.20,NIRCA Cross Country National Championships,race_results.php?race=844,"November 16, 2019",6k,F
38209,142.0,-,Columbia University,Mehta,Sukanya,SR,33:32.10,NIRCA Cross Country National Championships,race_results.php?race=844,"November 16, 2019",6k,F
38210,143.0,-,University of Maryland Baltimore County,Gregory,Katelyn,GRAD,33:46.40,NIRCA Cross Country National Championships,race_results.php?race=844,"November 16, 2019",6k,F
38211,144.0,-,George Mason University,Burris,Rebecca,SR,34:21.60,NIRCA Cross Country National Championships,race_results.php?race=844,"November 16, 2019",6k,F


Now I need to replace the NAs in score with 0s. For now I'm going to leave the NAs in Place, as I can't replace them with 0, and I'd need to check each on a case by case basis to see how to replace them.

In [54]:
comb.Score = comb.Score.replace(np.nan, 0)
comb.isna().sum()

Place         25
Score          0
Club           0
Last Name      0
First Name     0
Class          0
Result         0
race           0
url            0
date           0
distance       0
sex            0
dtype: int64

In addition to NAs in the Result category, there are 'DNS' or 'DNF' values for runners who failed to start or finish a race. Those need to be dropped as well

In [55]:
dns = comb[(comb["Result"] == "DNS")].index
dnf = comb[(comb["Result"] == "DNF")].index
comb.drop(dns, inplace = True)
comb.drop(dnf, inplace = True)
comb

,Place,Score,Club,Last Name,First Name,Class,Result,race,url,date,distance,sex
0,1.0,1.0,North Carolina State University,Chandler II,Max,ALUMN,26:29.00,NIRCA XC Regional Championship (Southeast),race_results.php?race=248,"October 24, 2015",8k,M
1,2.0,2.0,North Carolina State University,Baraldi,Robert,SR,26:30.00,NIRCA XC Regional Championship (Southeast),race_results.php?race=248,"October 24, 2015",8k,M
2,3.0,3.0,University of Tennessee,Brey,Nick,ALUMN,26:33.60,NIRCA XC Regional Championship (Southeast),race_results.php?race=248,"October 24, 2015",8k,M
3,4.0,4.0,North Carolina State University,Rees-Jones,Josh,ALUMN,26:46.70,NIRCA XC Regional Championship (Southeast),race_results.php?race=248,"October 24, 2015",8k,M
4,5.0,5.0,Duke University,Cardel,Patrick,ALUMN,26:55.90,NIRCA XC Regional Championship (Southeast),race_results.php?race=248,"October 24, 2015",8k,M
...,...,...,...,...,...,...,...,...,...,...,...,...
38208,141.0,-,Central Michigan University,Kosnak,Katie,GRAD,33:22.20,NIRCA Cross Country National Championships,race_results.php?race=844,"November 16, 2019",6k,F
38209,142.0,-,Columbia University,Mehta,Sukanya,SR,33:32.10,NIRCA Cross Country National Championships,race_results.php?race=844,"November 16, 2019",6k,F
38210,143.0,-,University of Maryland Baltimore County,Gregory,Katelyn,GRAD,33:46.40,NIRCA Cross Country National Championships,race_results.php?race=844,"November 16, 2019",6k,F
38211,144.0,-,George Mason University,Burris,Rebecca,SR,34:21.60,NIRCA Cross Country National Championships,race_results.php?race=844,"November 16, 2019",6k,F


The last thing I want to do to this dataset is convert the results to a float of seconds. Because this is club running, and not a rigorous organization like the NCAA, there will be times in all kinds of formats. As a result, I need to make a specialized function to handle it.

In [56]:
def to_seconds(time):
    # Handles various formats of times 
    # MM:SS.ss
    # MM.SS
    # MMm SSs
    # others soon to be found
    
    if ":" in time:
        # MM:SS.ss format
        spl = time.split(":")
        mins = int(spl[0])
        secs = int(spl[1].split(".")[0])
        if len(spl[1]) > 2:
            ms = int(spl[1].split(".")[0])
        else:
            ms = 0
    
    elif (" " in time):
        # MMm SSs.00 format
        # I want to meet the animal that decided to enter a time as 30m 1s.00
        spl = time.split(" ")
        mins = int(spl[0].split("m")[0])
        secs = int(spl[1].split(".")[0].split("s")[0])
        ms = 0
    
    elif "." in time:
        # MM.SS format
        spl = time.split(".")
        mins = int(spl[0])
        secs = int(spl[1])
        ms = 0
    
    return 60 * mins + secs + (ms / 100)

In [57]:
comb['Result'] = comb['Result'].apply(to_seconds)

In [58]:
comb

,Place,Score,Club,Last Name,First Name,Class,Result,race,url,date,distance,sex
0,1.0,1.0,North Carolina State University,Chandler II,Max,ALUMN,1589.29,NIRCA XC Regional Championship (Southeast),race_results.php?race=248,"October 24, 2015",8k,M
1,2.0,2.0,North Carolina State University,Baraldi,Robert,SR,1590.30,NIRCA XC Regional Championship (Southeast),race_results.php?race=248,"October 24, 2015",8k,M
2,3.0,3.0,University of Tennessee,Brey,Nick,ALUMN,1593.33,NIRCA XC Regional Championship (Southeast),race_results.php?race=248,"October 24, 2015",8k,M
3,4.0,4.0,North Carolina State University,Rees-Jones,Josh,ALUMN,1606.46,NIRCA XC Regional Championship (Southeast),race_results.php?race=248,"October 24, 2015",8k,M
4,5.0,5.0,Duke University,Cardel,Patrick,ALUMN,1615.55,NIRCA XC Regional Championship (Southeast),race_results.php?race=248,"October 24, 2015",8k,M
...,...,...,...,...,...,...,...,...,...,...,...,...
38208,141.0,-,Central Michigan University,Kosnak,Katie,GRAD,2002.22,NIRCA Cross Country National Championships,race_results.php?race=844,"November 16, 2019",6k,F
38209,142.0,-,Columbia University,Mehta,Sukanya,SR,2012.32,NIRCA Cross Country National Championships,race_results.php?race=844,"November 16, 2019",6k,F
38210,143.0,-,University of Maryland Baltimore County,Gregory,Katelyn,GRAD,2026.46,NIRCA Cross Country National Championships,race_results.php?race=844,"November 16, 2019",6k,F
38211,144.0,-,George Mason University,Burris,Rebecca,SR,2061.21,NIRCA Cross Country National Championships,race_results.php?race=844,"November 16, 2019",6k,F


Finally, I can save the dataset to a csv for upcoming analysis

In [59]:
comb.to_csv("nirca_historical_times.csv")

## Team Member Contributions

Caleb joined the group after I had already submitted this, so I did all the work in this notebook. 